# Simple simulations in R (HW4)

This is just an example of a quick way of throwing together a simulation model in R, [imperative](https://en.wikipedia.org/wiki/Imperative_programming) style (opposed to the functional approach in the event study example)

Encapsulating the population in a function is a nice first step: `getPopulation` returns a data frame with the specified population parameters. Using `gl` it is easy to create the firm factor with levels `1..N`


In [1]:
getPopulation = function(N, T, gamma, beta0 = .1, beta1 = -.2) {
  firm = gl(N, T)
  a = rnorm(N)[firm]
  prof = gamma * a + rnorm(N * T, mean = 0, sd = .25)
  lev = beta0 + beta1 * prof + a + rnorm(N * T)

  data.frame(time = rep(1:T, N), firm, a, prof, lev)           
}

In [2]:
head(getPopulation(100, 2, 0))

,time,firm,a,prof,lev
1,1,1,-0.993034,-0.4515975,0.4764365
2,2,1,-0.993034,0.2936457,-0.7107503
3,1,2,-0.5055087,0.4372172,1.665138
4,2,2,-0.5055087,-0.220917,-0.6538248
5,1,3,-0.7284925,0.422743,0.1045457
6,2,3,-0.7284925,-0.02529971,-0.08532578


First of all, since we are only interested in the regression coefficients, we can speed up the simulations quite simply, by avoiding instantiating a whole bunch of `lm` objects:

Remember that the OLS estimates are given by the formula $x = (A'A)^{-1} A'b$, or equivalently the `x` that solves
$A'Ax = A'b$, where the latter is a standard system of linear equations (i.e. $Bz=y$) that can be solved quickly by `solve()` (which in turn calls a FORTRAN [LAPACK](http://www.netlib.org/lapack/explore-3.1.1-html/dgesv.f.html) routine).

So the simple regression coefficients can be calculated with the `lstsq` function:

In [3]:
lstsq = function(A, b)
  solve(crossprod(A), crossprod(A, b))

In the case of the **fixed effects dummies**, for this example, it's easier to just leave this to `lm`,  The resulting `A` matrix will be [sparse](https://en.wikipedia.org/wiki/Sparse_matrix), and `solve()` is not a particular good way of speeding this up.

So, a quick way of encapsulating everything is to

* Create an empty matrix `coeffs` to hold the estimated coefficients from each run
* For `i` `1..N` generate a population, get the `ols` coefficient, if we want fixed effects (question a and b) get the `fe` coefficients as well
* return the mean of the coefficients, with simulated confidence intervals.

In [4]:
runSim = function(Nsim, N, T, gamma, FE = TRUE) {
  coeffs = matrix(0, nrow = Nsim, ncol = 2)
  
  for (i in 1:Nsim) {
    df = getPopulation(N, T, gamma)
    # ols = lm(lev ~ prof, df)$coefficients[2] #, the same as:
    ols = lstsq(cbind(df$prof, rep(1, N * T)), df$lev)[1]
    cfs = ols
    if (FE) {
      fe = lm(lev ~ prof + firm, df)$coefficients[2]
      cfs = cbind(cfs, fe)
    }
    coeffs[i, ] = cfs
  }
  out = rbind(ols = c(prof = mean(coeffs[, 1]),
                      quantile(coeffs[, 1], c(.025, .975))))
  if (FE)
    out = rbind(out, fe = c(prof = mean(coeffs[, 2]),
                            quantile(coeffs[, 2], c(.025, .975))))
 out 
}

which gives

#### Question a) and b)

In [5]:
Nsim = 1000
runSim(Nsim, 100, 2, 0.0)
runSim(Nsim, 100, 2, .7)

,prof,2.5%,97.5%
ols,-0.2005488,-0.9795816,0.5633022
fe,-0.2125852,-0.9937218,0.5935432


,prof,2.5%,97.5%
ols,1.0622605,0.8708411,1.2498709
fe,-0.1862344,-0.9774512,0.5866154


####  Question d)

In [6]:
runSim(Nsim, 50, 2, 0, FALSE)
runSim(Nsim, 1000, 2, 0, FALSE)

,prof,2.5%,97.5%
ols,-0.1850265,-1.2855690,0.9929154


,prof,2.5%,97.5%
ols,-0.19706206,-0.43672359,0.05357798


#### Question e)

In [7]:
runSim(Nsim, 100, 50, 0, FALSE)
runSim(Nsim, 100, 1000, 0, FALSE)

,prof,2.5%,97.5%
ols,-0.1941472,-0.3382455,-0.0436800


,prof,2.5%,97.5%
ols,-0.2001777,-0.2350773,-0.1659204
